In [ ]:
import random

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f

Prepare data

In [ ]:
df = pd.read_csv("Iris.csv")
df.head()

In [ ]:
df = df.drop(["Id"], axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
df.Species.unique()

In [ ]:
X = df.drop(["Species"], axis=1).values
Y = df["Species"]

In [ ]:
encoder = OneHotEncoder(sparse=False)
Y = encoder.fit_transform(Y.values.reshape(-1, 1))

In [ ]:
X_tensor = torch.from_numpy(X).type(torch.FloatTensor)
Y_tensor = torch.from_numpy(Y).type(torch.FloatTensor)

Define model

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.__input_layer = nn.Linear(input_dim, 200)
        self.__hidden_layer = nn.Linear(200, 100)
        self.__output_layer = nn.Linear(100, output_dim)

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        x = torch.transpose(x,0,1)

        # x = [batch size, height * width]

        h_1 = f.relu(self.__input_layer(x))

        # h_1 = [batch size, 250]

        h_2 = f.relu(self.__hidden_layer(h_1))

        # h_2 = [batch size, 100]

        y_pred = self.__output_layer(h_2)

        # y_pred = [batch size, output dim]

        return y_pred, h_2

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.linear_stack = nn.Sequential(
            nn.Linear(input_dim, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, output_dim)
        )

    def forward(self, x):
        return self.linear_stack(x)

Prepare training and test procedures

In [ ]:
INPUT_DIM = 4
OUTPUT_DIM = 3

model = Net(INPUT_DIM, OUTPUT_DIM)

# choose Stochastic Gradient Descent for optimization
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    num_correct_pred = top_pred.eq(y.view_as(top_pred)).sum()
    accuracy = num_correct_pred / y.shape[0]
    return accuracy


def train(model, x_train, y_train, loss_func, device):
    train_loss,train_accuracy=0.0,0.0
    model.train()

    for i,data in enumerate(x_train):
        data,y_train[i] = data.to(device),y_train[i].to(device)

        optimizer.zero_grad()

        result = list(model(data))
        result[0] = result[0]
        loss = loss_func(result[0],y_train[i])

        loss.backward()
        optimizer.step()

        accuracy = calculate_accuracy(result[0],y_train[i])
        train_loss += loss.item() 
        train_accuracy+= accuracy.item()

    return train_loss/ len(y_train), train_accuracy / len(y_train)    
   


def test(model,x_test,y_test, loss_func, device):
    test_loss, test_accuracy = 0.0, 0
    model.eval()

    for i,data in enumerate(x_test):
        data,y_test[i] = data.to(device),y_test[i].to(device)

        result = list(model(data))
        result[0] = result[0].view(3,-1)
        loss=loss_func(result[0],y_test[i].view(3,-1))

        accuracy = calculate_accuracy(result[0],y_train[i])
        test_loss += loss.item() 
        test_accuracy+= accuracy.item()

       

    return test_loss/ len(y_train), test_accuracy / len(y_train)  

def kfold(model,X, Y, loss_func, device, k):
    kf = KFold(n_splits = k)
    kf.get_n_splits(X)

    fold=0
    for train_id, test_id in kf.split(X):

        print(f"Fold {fold+1}")    
        x_train, x_test = X[train_id], X[test_id]
        y_train, y_test = Y[train_id], Y[test_id]

        for epoch in range(num_epochs):    
            train_loss, train_accuracy = train(model,x_train,y_train,loss_func,device)
            test_loss, test_accuracy = train(model,x_test,y_test,loss_func,device)
        print(f"Epoch:{epoch+1}/{num_epochs} Training loss:{train_loss} Training accuracy:{train_accuracy} Test loss:{test_loss} Test accuracy:{test_accuracy} \n") 

Model 1: Loss function: Cross Entropy Loss, Layers: 2

In [ ]:
crossEntropyLoss = nn.CrossEntropyLoss

# choose GPU for computations if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_1 = model.to(device)
#loss_func = loss_func.to(device)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_tensor, Y_tensor)

best_valid_loss = float('inf')

num_epochs = 12
for epoch in range(num_epochs):
    train_loss,train_accuracy = train(model =model_1,x_train = x_train,y_train = y_train,loss_func=crossEntropyLoss,device=device)
    test_loss, test_accuracy = test(model = model_1,x_test = x_test,y_test = y_test,loss_func=crossEntropyLoss,device=device)

    if test_loss < best_valid_loss:
        best_valid_loss = test_loss

    print(f"Epoch:{epoch+1}/{num_epochs} Training loss:{train_loss} Training accuracy:{train_accuracy} Test loss:{test_loss} Test accuracy:{test_accuracy} \n")

In [ ]:

#kfold(model=model_1,X=X_tensor,Y=Y_tensor,loss_func= crossEntropyLoss,device=device,k=5)